# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification
<br>Video disponible en: https://drive.google.com/file/d/16a9lxTxlBlv13Bh5LxRoAMB8-DwKilv2/view?usp=sharing
<br>Dataset, csv y .ows en: https://drive.google.com/drive/folders/1oqmh6m7Xvt7Y1qrx2FHEz0nlC74QCSJE?usp=sharing

## 1. DATA READ


In [5]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")

Dataset.keys()

Index(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       ...
       'n2044', 'n2045', 'n2046', 'n2047', 'category', 'image name', 'image',
       'size', 'width', 'height'],
      dtype='object', length=2054)

In [6]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
Dataset['category'] = Dataset['category'].replace(['Camelia', 'Dahlia', 'Hydrangea', 'Lilies', 'Orchids', 'Peony', 'Ranunculus', 'Roses', 'Sunflowers', 'Tulips'], [0,1,2,3,4,5,6,7,8,9])
DataFrameFlowers = pd.DataFrame(Dataset)

## 2. Data Preprocessing

In [7]:
X = DataFrameFlowers.drop(['category'], axis = 1)
Y = DataFrameFlowers['category']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## Model #1 Logistic Regression

### 3. Model Creation

In [8]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)
LR_better = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l2", solver="lbfgs", tol=1e-4)

### 4. Training: Adjust Model with Historic Data

In [9]:
LR.fit(X_train, y_train)
LR_better.fit(X_train, y_train)

C:\Users\Alex\anaconda3\envs\ML_DL\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.24875621890547264)

### 5. Prediction for New Data

In [10]:
y_predictLR = LR.predict(X_test)
y_predictLR_better = LR_better.predict(X_test)

## Model #2 Convolutional Neural Network (CNN)

In [11]:
#Aqui haremos las variables especialmente para el modelo CNN, ya que se necesita estar en 3 dimensiones
#Para training 201 x 2048 x 1, para test 51 x 2048 x 1
X_trainCNN = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_testCNN = X_test.values.reshape(X_test.shape[0], X_test.shape[1] ,1)

In [12]:
X_train.shape

(201, 2048)

### 3. Model Creation

In [13]:
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import  Dense, Conv1D, Flatten, MaxPooling1D, Dropout

#Para coincidir tamaños, se convierten las etiquetas en "one hot vectors"
y_trainCNN = keras.utils.to_categorical(np.asarray(y_train.factorize()[0]))
y_testCNN = keras.utils.to_categorical(np.asarray(y_test.factorize()[0]))

CNN = Sequential()
CNN.add(Conv1D(128, 3, activation='relu', input_shape=(2048,1)))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
CNN.add(Dense(10, activation='softmax'))

CNN.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [14]:
CNN_better = Sequential()
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(2048,1)))
CNN_better.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
CNN_better.add(Dropout(0.7))
CNN_better.add(MaxPooling1D(pool_size=2))
CNN_better.add(Flatten())
CNN_better.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
CNN_better.add(Dense(10, activation='softmax'))

CNN_better.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [15]:
y_trainCNN.shape

(201, 10)

### 4. Training: Adjust Model with Historic Data

In [16]:
CNN_H = CNN.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 2s 285ms/step - loss: 2.8052 - accuracy: 0.1045 - val_loss: 2.3070 - val_accuracy: 0.1373
Epoch 2/10
7/7 [==============================] - 2s 262ms/step - loss: 2.2466 - accuracy: 0.1891 - val_loss: 2.2323 - val_accuracy: 0.1569
Epoch 3/10
7/7 [==============================] - 2s 259ms/step - loss: 1.9459 - accuracy: 0.3731 - val_loss: 2.1780 - val_accuracy: 0.3137
Epoch 4/10
7/7 [==============================] - 2s 257ms/step - loss: 1.4618 - accuracy: 0.5174 - val_loss: 2.2406 - val_accuracy: 0.3529
Epoch 5/10
7/7 [==============================] - 2s 256ms/step - loss: 0.8847 - accuracy: 0.7214 - val_loss: 3.4667 - val_accuracy: 0.2549
Epoch 6/10
7/7 [==============================] - 2s 279ms/step - loss: 0.4361 - accuracy: 0.8209 - val_loss: 3.4237 - val_accuracy: 0.4314
Epoch 7/10
7/7 [==============================] - 2s 265ms/step - loss: 0.3809 - accuracy: 0.8905 - val_loss: 4.2507 - val_accuracy: 0.3137
Epoch 8/10
7/7 [====

In [17]:
CNN_HBetter = CNN_better.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 1s 156ms/step - loss: 3.4883 - accuracy: 0.2090 - val_loss: 2.2394 - val_accuracy: 0.1961
Epoch 2/10
7/7 [==============================] - 1s 145ms/step - loss: 2.1068 - accuracy: 0.3085 - val_loss: 2.1486 - val_accuracy: 0.2353
Epoch 3/10
7/7 [==============================] - 1s 145ms/step - loss: 1.8411 - accuracy: 0.4826 - val_loss: 2.0929 - val_accuracy: 0.3725
Epoch 4/10
7/7 [==============================] - 1s 152ms/step - loss: 1.2651 - accuracy: 0.7015 - val_loss: 2.1791 - val_accuracy: 0.4118
Epoch 5/10
7/7 [==============================] - 1s 143ms/step - loss: 0.6391 - accuracy: 0.8259 - val_loss: 2.7298 - val_accuracy: 0.3333
Epoch 6/10
7/7 [==============================] - 1s 136ms/step - loss: 0.3755 - accuracy: 0.8856 - val_loss: 3.5252 - val_accuracy: 0.2353
Epoch 7/10
7/7 [==============================] - 1s 137ms/step - loss: 0.2971 - accuracy: 0.9005 - val_loss: 3.3977 - val_accuracy: 0.4118
Epoch 8/10
7/7 [====

### 5. Prediction for New Data

In [18]:
y_predictCNN = CNN.predict_classes(X_testCNN)
y_predictCNN_better = CNN_better.predict_classes(X_testCNN)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


## Model #3 Support Vector Machine (SVM)

### 3. Model Creation

In [19]:
from sklearn import svm
SVM = svm.SVC(kernel='poly')
SVM_better = svm.SVC(kernel='linear')

### 4. Training: Adjust Model with Historic Data

In [20]:
SVM.fit(X_train, y_train)
SVM_better.fit(X_train, y_train)

SVC(kernel='linear')

### 5. Prediction for New Data

In [21]:
y_predictSVM = SVM.predict(X_test)
y_predictSVM_better = SVM_better.predict(X_test)

## Model #4 Random Forest

### 3. Model Creation

In [22]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=500)
RFC_better = RandomForestClassifier(n_estimators= 400,bootstrap= False)

### 4. Training: Adjust Model with Historic Data

In [23]:
RFC.fit(X_train, y_train)
RFC_better.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_features='sqrt', n_estimators=400)

### 5. Prediction for New Data

In [24]:
y_predictRF = RFC.predict(X_test)
y_predictRF_better = RFC_better.predict(X_test)

## 6. Visualization of Results

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### Model #1 Logistic Regression

In [26]:
CV_LR = cross_val_score(LR, X_train, y_train, cv=3, scoring = "accuracy")
CV_LR_better = cross_val_score(LR_better, X_train, y_train, cv=3, scoring = "accuracy")

C:\Users\Alex\anaconda3\envs\ML_DL\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Alex\anaconda3\envs\ML_DL\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

In [27]:
AS_LR = accuracy_score(y_test,y_predictLR)
AS_LR_better = accuracy_score(y_test,y_predictLR_better)

### Model #2 Support Vector Machine (SVM)

In [28]:
CV_SVM = cross_val_score(SVM, X_train, y_train, cv=3, scoring = "accuracy")
CV_SVM_better = cross_val_score(SVM_better, X_train, y_train, cv=3, scoring = "accuracy")

In [29]:
AS_SVM = accuracy_score(y_test,y_predictSVM)
AS_SVM_better = accuracy_score(y_test,y_predictSVM_better)

### Model #3 Random Forest

In [30]:
CV_RF = cross_val_score(RFC, X_train, y_train, cv=3, scoring = "accuracy")
CV_RF_better = cross_val_score(RFC_better, X_train, y_train, cv=3, scoring = "accuracy")

In [31]:
AS_RF = accuracy_score(y_test,y_predictRF)
AS_RF_better = accuracy_score(y_test,y_predictRF_better)

### Model #5 Convolutional Neural Network (CNN)

In [32]:
CV_CNN = CNN.evaluate(X_trainCNN, y_trainCNN)
CV_CNN_better = CNN_better.evaluate(X_trainCNN, y_trainCNN)

7/7 [==============================] - 0s 20ms/step - loss: 0.0540 - accuracy: 0.9851


In [33]:
AS_CNN = CNN.evaluate(X_testCNN, y_testCNN)
AS_CNN_better = CNN_better.evaluate(X_testCNN, y_testCNN)

2/2 [==============================] - 0s 8ms/step - loss: 4.0452 - accuracy: 0.3725


In [34]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better", "RF_better", "CNN_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better, AS_RF_better, AS_CNN_better[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better, CV_RF_better, [CV_CNN_better[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

,Accuracy Scores,CV AVG,CV #1,CV #2,CV #3
LR,0.666667,0.597015,0.597015,0.686567,0.746269
SVM,0.588235,0.716418,0.716418,0.716418,0.746269
RF,0.647059,0.671642,0.671642,0.716418,0.701493
CNN,0.274510,0.985075,0.985075,-,-
LR_better,0.745098,0.716418,0.716418,0.731343,0.761194
SVM_better,0.686275,0.746269,0.746269,0.746269,0.731343
RF_better,0.764706,0.656716,0.656716,0.731343,0.656716
CNN_better,0.372549,0.985075,0.985075,-,-


## Confusion Matrix


In [35]:
from sklearn.metrics import confusion_matrix
cmLR=confusion_matrix(y_test,y_predictLR)
print(cmLR)

print("--------------------------------")
cmLR_better=confusion_matrix(y_test,y_predictLR_better)
print(cmLR_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [1 0 2 1 1 0 0 0 0 0]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 0 0 0 1 8 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 1 0 0 0 1 0 4]]
--------------------------------
[[3 0 0 0 1 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 1 0 0 2 5 0 1]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 1 0 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de logistic Regression. 

In [36]:
cmCNN=confusion_matrix(y_test,y_predictCNN)
print(cmCNN)

print("--------------------------------")
cmCNN_better=confusion_matrix(y_test,y_predictCNN_better)
print(cmCNN_better)

[[0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0]
 [0 0 4 0 0 1 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 1 0 0 1 0]
 [0 0 1 0 1 1 0 1 0 3]
 [2 1 0 1 0 0 0 0 1 2]
 [4 2 0 2 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 1 0 0]
 [1 1 0 0 2 0 0 0 2 0]]
--------------------------------
[[0 1 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 2 1 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 2 0]
 [0 0 0 0 3 0 0 0 0 4]
 [3 1 0 0 0 0 0 0 1 2]
 [3 1 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0]
 [1 1 0 0 4 0 0 0 0 0]]


En las matrices de confusión de CNN se puede observar que no hubo muchas flores clasificadas correctamente, pues el accuracy de los modelos fue también bajo. No hubo mucha variación entre los resultados de cada una, puesto que su diagonal fue similar.

In [37]:
cmSVM=confusion_matrix(y_test,y_predictSVM)
print(cmSVM)

print("--------------------------------")
cmSVM_better=confusion_matrix(y_test,y_predictSVM_better)
print(cmSVM_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 0 4]]
--------------------------------
[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 1 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de Support Vector Machine. 

In [38]:
cmRF=confusion_matrix(y_test,y_predictRF)
print(cmRF)

print("--------------------------------")
cmRF_better=confusion_matrix(y_test,y_predictRF_better)
print(cmRF_better)

[[3 0 0 0 1 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 2 0 0 0 0 2]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 1 0 0 2 5 0 1]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 1 0 0 1 0 0 4]]
--------------------------------
[[4 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 3 0 0 0 0 1]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 1 1 0 5 0 0 0]
 [0 0 0 1 0 0 1 7 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 1 0 0 0 0 0 5]]


Se puede observar que Random Forest en ambos modelos clasificó correctamente la mayoría de los datos, aunque también se aprecia una mejora del primer al segundo modelo al tener menos valores fuera de la diagonal.

## Preguntas

### ¿Con qué tipo de flor se lograron los mejores resultados?

In [86]:
def bestFlowerPerModel(cm):
    diagonal = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        diagonal[i] = cm[i][i]

    sumRow = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        for j in range(10):
            sumRow[i] += cm[i][j]

    avg = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        avg[i] = diagonal[i]/sumRow[i]

    score = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        score[i] = avg[i]*(sumRow[i]/51)

    highest = 0
    pos = 0
    for i in range(10):
        if score[i] > highest:
            highest = score[i]
            pos = i

    return "The best flower is: " + str(pos) + " with a score of " + str(highest)

print(bestFlowerPerModel(cmLR))
print(bestFlowerPerModel(cmLR_better))
print(bestFlowerPerModel(cmCNN))
print(bestFlowerPerModel(cmCNN_better))
print(bestFlowerPerModel(cmSVM))
print(bestFlowerPerModel(cmSVM_better))
print(bestFlowerPerModel(cmRF))
print(bestFlowerPerModel(cmRF_better))

The best flower is: 7 with a score of 0.1568627450980392
The best flower is: 5 with a score of 0.09803921568627452
The best flower is: 2 with a score of 0.0784313725490196
The best flower is: 2 with a score of 0.09803921568627451
The best flower is: 2 with a score of 0.0784313725490196
The best flower is: 5 with a score of 0.09803921568627452
The best flower is: 7 with a score of 0.09803921568627452
The best flower is: 7 with a score of 0.13725490196078433


Podemos observar que las flores mejor clasificadas fueron las Hydrangeas (2) y las Rosas (7)

### ¿Cuál flor fue la mas difícil de clasificar?

In [85]:
def worstFlowerPerModel(cm):
    diagonal = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        diagonal[i] = cm[i][i]

    sumRow = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        for j in range(10):
            sumRow[i] += cm[i][j]

    avg = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        avg[i] = diagonal[i]/sumRow[i]

    score = [0,0,0,0,0,0,0,0,0,0]
    for i in range(10):
        score[i] = avg[i]*(sumRow[i]/51)

    lowest = score[0]
    pos = 0
    for i in range(10):
        if score[i] < lowest:
            highest = score[i]
            pos = i

    return "The worst flower is: " + str(pos) + " with a score of " + str(lowest)

print(worstFlowerPerModel(cmLR))
print(worstFlowerPerModel(cmLR_better))
print(worstFlowerPerModel(cmCNN))
print(worstFlowerPerModel(cmCNN_better))
print(worstFlowerPerModel(cmSVM))
print(worstFlowerPerModel(cmSVM_better))
print(worstFlowerPerModel(cmRF))
print(worstFlowerPerModel(cmRF_better))

The worst flower is: 8 with a score of 0.058823529411764705
The worst flower is: 8 with a score of 0.058823529411764705
The worst flower is: 0 with a score of 0.0
The worst flower is: 0 with a score of 0.0
The worst flower is: 8 with a score of 0.058823529411764705
The worst flower is: 8 with a score of 0.058823529411764705
The worst flower is: 8 with a score of 0.058823529411764705
The worst flower is: 8 with a score of 0.0784313725490196


Aquí vemos que la flor peor categorizada evidentemente fue el girasol (8)

### ¿Cuál modelo obtuvo los mejores resultados?

In [89]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better", "RF_better", "CNN_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better, AS_RF_better, AS_CNN_better[1]]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better, CV_RF_better, [CV_CNN_better[1],'-','-']]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas, referenciando (10)
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales, referenciando (11)
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

,Accuracy Scores,CV AVG,CV #1,CV #2,CV #3
LR,0.666667,0.597015,0.597015,0.686567,0.746269
SVM,0.588235,0.716418,0.716418,0.716418,0.746269
RF,0.647059,0.671642,0.671642,0.716418,0.701493
CNN,0.274510,0.985075,0.985075,-,-
LR_better,0.745098,0.716418,0.716418,0.731343,0.761194
SVM_better,0.686275,0.746269,0.746269,0.746269,0.731343
RF_better,0.764706,0.656716,0.656716,0.731343,0.656716
CNN_better,0.372549,0.985075,0.985075,-,-


El modelo con mejores resultados fue el RF_better
En segundo tenemos a LR_better, con una diferencia muy pequeña

### ¿Cómo cree que pueda mejorar los resultados obtenidos?

La mejor manera de mejorar nuestros resultados sería incrementar el tamaño de nuestro dataset, ya que tenemos muy pocos datos
Otra manera de mejorar los resultados podría ser continuar ajustando los parámetros de los modelos hasta encontrar algunos que den resultados aun mejores, o incluso cambiar de modelos que se adapten más a los datos.

### ¿Cómo varían los resultados entre el set de datos de entrenamiento y el de pruebas?

En la mayoría de los modelos los resultados de entrenamiento y de pruebas son muy similares.
Sin embargo, en el modelo de CNN observamos resultados de entrenamiento muy altos, pero en pruebas resultados muy bajos
Esto nos indica que ocurrió overfitting, a pesar de que utilizamos dropout para intentar contrarrestar este efecto

## References

>_Se basó en apuntes de Regresión Lineal y teoría del curso para este proyecto._

_Para todos los modelos excepto CNN_:
<br>
1. Cournapeau, D. (2021). scikit-learn, Machine Learning in Python. Retrieved from: https://scikit-learn.org/stable/


 _Para CNN_:  
 
2. Chollet, F. (2022). Clasificacion Básica: Predecir una imagen de moda. Retrieved from: https://www.tensorflow.org/tutorials/keras/classification?hl=es-419#hacer_predicciones

3. TensorFlow. (2022). Red neuronal convolucional (CNN). Retrieved from: https://www.tensorflow.org/tutorials/images/cnn
4. Ruizendaal, R. (2017).Deep Learning #3: More on CNNs & Handling Overfitting. Retrieved from: https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d 
5. Brownlee, J. (2020). 1D Convolutional Neural Network Models for Human Activity Recognition. Retrieved from: https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/

6. Pandas. (2022). DataFrame. Retrieved from: https://pandas.pydata.org/docs/reference/frame.html